In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [2]:
import sys
sys.path.insert(0, r'C:/Users/Angie Michaiel/Documents/GitHub/FreelyMovingEphys')
from util.paths import find
from project_analysis.ephys.ephys_utils import load_ephys
from project_analysis.doi.doi_utils import doi_saline_label
from project_analysis.doi.doi_utils import pre_post_label
from project_analysis.doi.doi_utils import label_doi_conditions

In [3]:
# load data from days of recordings 
data_path = r'\\kraken\Z\freely_moving_ephys'
dates = ['021621','021721']
data = load_ephys(data_path,dates)


In [4]:
#Dictonary of conditions
conditions = {}
conditions['doi_saline'] = {'doi_dates':['021621'],'saline_dates':['021721']}
conditions['pre_post'] = {'pre_dates':['021621'],'post_dates':['021721']}
print(conditions)



{'doi_saline': {'doi_dates': ['021621'], 'saline_dates': ['021721']}, 'pre_post': {'pre_dates': ['021621'], 'post_dates': ['021721']}}


In [5]:
# Add conditions to data_dataframe 
pd.set_option('display.max_rows', None)
conditions_df = label_doi_conditions(data,conditions)


In [6]:
goodcells_df = conditions_df.loc[conditions_df['KSLabel']=='good']
goodcells_df= goodcells_df[:-1]
print(goodcells_df)


     Amplitude  ContamPct KSLabel         amp  ch  depth         fr group  \
5       1538.0        1.3    good   96.000031   4    676  15.167560  good   
8       1205.0        8.3    good  109.158997   9    551  12.648416  good   
17      1310.0        3.6    good   97.813812  20    276  33.508521  good   
21      2075.1        0.5    good  130.199249  24    176  38.572274  good   
49      1269.6        8.5    good   48.976822  45    451   0.459819  good   
56       963.2        9.9    good   56.565620  52    276  12.041113  good   
57      1346.6        0.3    good   74.077042  54    226   4.383297  good   
59      1463.2        1.3    good   72.136810  56    176  14.196847  good   
61      2023.1        8.2    good   89.483780  57    151   0.729028  good   
67      1237.3        1.1    good   79.985680  62     26   3.818090  good   
76      1285.4        3.0    good   57.095936  51    301   1.115547  good   
77      1076.4        0.0    good   74.696571  45    451   0.587874  good   

In [7]:
goodcells_df['ContamPct']

5      1.3
8      8.3
17     3.6
21     0.5
49     8.5
56     9.9
57     0.3
59     1.3
61     8.2
67     1.1
76     3.0
77     0.0
81     0.8
55     3.3
56     3.6
107    0.0
Name: ContamPct, dtype: float64

In [8]:
#Add column of Total time of recordings as total seconds
goodcells_df['Tsec_fm1'] = goodcells_df['fm1_spikeTraw'].str[-1]
goodcells_df['Tsec_fm2'] = goodcells_df['fm2_spikeTraw'].str[-1]
goodcells_df['Tsec_hf1'] = goodcells_df['hf1_wn_spikeTraw'].str[-1]
goodcells_df['Tsec_hf8'] = goodcells_df['hf8_wn_spikeTraw'].str[-1]
goodcells_df['Tsec_hf2'] = goodcells_df['hf2_sprasenoiseflash_spikeTraw'].str[-1]
goodcells_df['Tsec_hf7'] = goodcells_df['hf7_sprasenoiseflash_spikeTraw'].str[-1]
goodcells_df['Tsec_hf3'] = goodcells_df['hf3_gratings_spikeTraw'].str[-1]
goodcells_df['Tsec_hf6'] = goodcells_df['hf6_gratings_spikeTraw'].str[-1]



In [9]:
#Calculate mean firing rate
goodcells_df['meanfr_fm1'] = goodcells_df['fm1_n_spikes']/goodcells_df['Tsec_fm1']
goodcells_df['meanfr_fm2'] = goodcells_df['fm2_n_spikes']/goodcells_df['Tsec_fm2']
goodcells_df['meanfr_hf1'] = goodcells_df['hf1_wn_n_spikes']/goodcells_df['Tsec_hf1']
goodcells_df['meanfr_hf8'] = goodcells_df['hf8_wn_n_spikes']/goodcells_df['Tsec_hf8']
goodcells_df['meanfr_hf2'] = goodcells_df['hf2_sprasenoiseflash_n_spikes']/goodcells_df['Tsec_hf2']
goodcells_df['meanfr_hf7'] = goodcells_df['hf7_sprasenoiseflash_n_spikes']/goodcells_df['Tsec_hf7']
goodcells_df['meanfr_hf3'] = goodcells_df['hf3_gratings_n_spikes']/goodcells_df['Tsec_hf3']
goodcells_df['meanfr_hf6'] = goodcells_df['hf6_gratings_n_spikes']/goodcells_df['Tsec_hf6']


In [10]:
goodcells_df.dtypes

Amplitude                                                 float64
ContamPct                                                 float64
KSLabel                                                    object
amp                                                       float64
ch                                                          int64
depth                                                       int64
fr                                                        float64
group                                                      object
fm1_n_spikes                                                int64
sh                                                          int64
waveform                                                   object
fm1_spikeT                                                 object
t0                                                        float64
fm1_spikeTraw                                              object
fm1_rate                                                   object
fm1_c_rang

In [20]:
#Dataframes of DOI and saline groups
doi_df = goodcells_df.loc[goodcells_df['hf7_sprasenoiseflashdoi/saline']=='doi']
#print(doi_df)
saline_df = goodcells_df.loc[goodcells_df['hf7_sprasenoiseflashdoi/saline']=='saline']
#print(saline_df)

    Amplitude  ContamPct KSLabel         amp  ch  depth         fr group  \
5      1538.0        1.3    good   96.000031   4    676  15.167560  good   
8      1205.0        8.3    good  109.158997   9    551  12.648416  good   
17     1310.0        3.6    good   97.813812  20    276  33.508521  good   
21     2075.1        0.5    good  130.199249  24    176  38.572274  good   
49     1269.6        8.5    good   48.976822  45    451   0.459819  good   
56      963.2        9.9    good   56.565620  52    276  12.041113  good   
57     1346.6        0.3    good   74.077042  54    226   4.383297  good   
59     1463.2        1.3    good   72.136810  56    176  14.196847  good   
61     2023.1        8.2    good   89.483780  57    151   0.729028  good   
67     1237.3        1.1    good   79.985680  62     26   3.818090  good   
76     1285.4        3.0    good   57.095936  51    301   1.115547  good   
77     1076.4        0.0    good   74.696571  45    451   0.587874  good   
81      813.

In [21]:
doi_df[doi_df.columns[-8:]] 

,meanfr_fm1,meanfr_fm2,meanfr_hf1,meanfr_hf8,meanfr_hf2,meanfr_hf7,meanfr_hf3,meanfr_hf6
5,55.500847,51.795030,162.064884,156.683871,310.221679,319.881031,286.431261,313.050019
8,46.282703,43.344184,135.124581,131.316659,258.703894,267.225264,238.886543,267.606289
17,122.616402,114.426291,357.982144,346.161051,685.447538,706.694198,632.846377,691.589003
21,141.142866,131.717056,412.066188,398.475026,788.995291,812.903646,728.424450,796.149328
49,1.682540,1.573344,5.037753,4.751039,9.435589,9.687089,8.721618,9.490214
56,44.060699,41.118357,128.637242,124.390619,246.318214,253.751143,227.392925,248.541991
57,16.039406,14.980947,46.870497,45.286203,89.921215,92.586442,82.819911,90.555665
59,51.950689,48.481523,151.668578,146.657573,290.392973,299.420053,268.121871,293.126882
61,2.677697,2.490442,7.837416,7.534999,14.910543,15.474654,14.151667,15.051306
67,14.007882,13.038829,40.810153,39.512583,78.119691,80.588413,72.397048,79.136797
